In [4]:
import pandas as pd 
import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier as RFC

In [2]:
def sift_features(images):
    sift_vectors = {}
    descriptor_list = []
    sift = cv2.SIFT_create()
    for key,value in images.items():
        features = []
        for img in value:
            kp, des = sift.detectAndCompute(img,None)
           
            
            descriptor_list.extend(des)
            features.append(des)
        sift_vectors[key] = features
    return [descriptor_list, sift_vectors]

def find_index(feature, centers):
    return np.argmin(np.sum((centers - feature) **2, axis = 1))

centers = np.load('centers.npy')

In [5]:
data = pd.read_csv('train.csv')
data.drop(columns='Unnamed: 0',inplace = True)
X = np.array(data.iloc[:,:-1])
y = np.array(data['y'])
model = RFC()
model.fit(X, y)

marker = cv2.imread('marker.jpg')
marker = cv2.resize(marker, (960, 1280))
repl = cv2.imread('repl.jpg')
repl = cv2.resize(repl, (960, 1280))

In [15]:
sift = cv2.SIFT_create()
kp_marker, des_marker = sift.detectAndCompute(marker, None)

cap = cv2.VideoCapture('video.mp4')

width = 960
height = 1280

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
CADR_PER_SEC = 24
MIN_MATCH_COUNT = 10
out = cv2.VideoWriter('result.mp4', fourcc, CADR_PER_SEC,  (width, height))

In [16]:
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        iter+=1
        frame = cv2.resize(frame,(960, 1280))
        frame_t = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(frame_t,None)
        histogram = np.zeros(len(centers))
        res = 2
        if des is not None:
            for each_feature in des:
                     ind = find_index(each_feature, centers)
                     histogram[ind] += 1
            #print(histogram)
            res = model.predict(histogram.reshape(1, -1))[0]
        if res == 1:
            bf = cv2.BFMatcher()
            matches = bf.knnMatch(des_marker, des, k=2)
    
            good = []
            for m, n in matches:
                if m.distance / n.distance <= 0.8:
                    good.append(m)

            if len(good)>=MIN_MATCH_COUNT:
                src_pts = np.float32([ kp_marker[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
                dst_pts = np.float32([ kp[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

                h,w,_ = marker.shape
                pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts,M)
                
                result = cv2.warpPerspective(repl, M, (repl.shape[1], repl.shape[0]))

                img2gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
                ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
                mask_inv = cv2.bitwise_not(mask)
                img1_bg = cv2.bitwise_and(frame, frame, mask=mask_inv)
                img2_fg = cv2.bitwise_and(result, result, mask=mask)
                frame = cv2.add(img1_bg, img2_fg)
                frame = cv2.polylines(frame,[np.int32(dst)],True,255,10, cv2.LINE_AA)
        
        
        out.write(frame)
        #cv2.imshow('result', frame)
        if cv2.waitKey(1000//CADR_PER_SEC) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.waitKey(0)
cv2.destroyAllWindows()